# BaiscTick: Create Cluster: RDB
This notebook will create (start) an RDB cluster on a named database.

In [1]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env_kdb_1 import *

# Cluster names and database
from basictick_setup import *

SEC_THREADS='8'

# Cluster Settings

CODEBASE="basictick"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.xlarge'}
SAVEDOWN_CONFIG={ 'type': 'SDS01', 'size':400 }
DATABASE_CONFIG=[{ 'databaseName': DB_NAME }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }
INIT_SCRIPT=f'{CODEBASE}/rdbmkdb.q'

CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'dbname', 'value': DB_NAME}, 
    { 'key': 'codebase', 'value': CODEBASE}, 
    { 'key': 'tphostfile', 'value': TP_HOSTFILE}, 
#    {'key': 'tp', 'value': TP }, 
]

In [3]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## Check Database

In [4]:
note_str = ""

try:
    c_set_list = client.list_kx_changesets(environmentId=ENV_ID, databaseName=DB_NAME)['kxChangesets']
except:
    note_str = "<<Could not get changesets>>"

print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"Changeset ({c['status']}): {c_set_id}: Created: {c['createdTimestamp']}")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: basictickdb, Changesets: 1 
Changeset (COMPLETED): 4MRqXexrXcIiOdRfJZbOJQ: Created: 2023-06-19 23:27:12.089000+00:00


changeType,s3Path,dbPath
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.23/,/2023.04.23/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.15/,/2023.04.15/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.14/,/2023.04.14/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.22/,/2023.04.22/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.18/,/2023.04.18/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.20/,/2023.04.20/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.16/,/2023.04.16/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.17/,/2023.04.17/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.21/,/2023.04.21/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2023.04.19/,/2023.04.19/


# Deploy code

In [5]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
else:
    cp = f"""
zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
    
# execute the S3 copy
os.system(cp)

updating: basictick/ (stored 0%)
updating: basictick/connectmkdb.q (deflated 63%)
updating: basictick/example.schema.q (deflated 12%)
updating: basictick/feedmkdb.q (deflated 53%)
updating: basictick/funcDownHandle.q (deflated 33%)
updating: basictick/gwmkdbcluster.q (deflated 61%)
updating: basictick/hdbmkdb.q (deflated 52%)
updating: basictick/loadDep.q (deflated 57%)
updating: basictick/query.q (deflated 19%)
updating: basictick/rdbmkdb.q (deflated 57%)
updating: basictick/tickerplant2 (stored 0%)
updating: basictick/tickerplant3 (stored 0%)
updating: basictick/tp.q (deflated 52%)
updating: basictick/.ipynb_checkpoints/ (stored 0%)
updating: basictick/.ipynb_checkpoints/tickerplant3-checkpoint (stored 0%)
updating: basictick/.ipynb_checkpoints/tickerplant-checkpoint (stored 0%)
updating: basictick/tickerplant (stored 0%)
updating: basictick/.ipynb_checkpoints/rdbmkdb-checkpoint.q (deflated 57%)
updating: basictick/.ipynb_checkpoints/gwmkdbcluster-checkpoint.q (deflated 61%)
upload: 

0

## Create Cluster

In [6]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=RDB_CLUSTER_NAME,
    clusterType='RDB',
    releaseLabel = '1.0',
    databases=DATABASE_CONFIG,
    capacityConfiguration=CAPACITY_CONFIG,
    savedownStorageConfiguration=SAVEDOWN_CONFIG,
    clusterDescription="Created with create_RDB notebook",
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    commandLineArguments=CMD_ARGS,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [7]:
resp

{'ResponseMetadata': {'RequestId': '10090d5c-30b8-466d-916a-b960065e50b2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1114',
   'connection': 'keep-alive',
   'date': 'Mon, 19 Jun 2023 23:29:06 GMT',
   'x-amzn-requestid': '10090d5c-30b8-466d-916a-b960065e50b2',
   'x-amz-apigw-id': 'GyitkGXIoAMFY5A=',
   'x-amzn-trace-id': 'Root=1-6490e4bc-17f3004e0dd9a4b5595ef837',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 eb30892e0d329aead17e943e1b1cc5ae.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'SOycjEvIWaNf-0_A4JSfbBBbBt1i3gb9NlzKu3QpHft2qYW-uAGWkg=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'RDB_basictickdb',
 'clusterType': 'RDB',
 'databases': [{'databaseName': 'basictickdb',
   'cacheConfigurations': [],
   'changesetId': '4MRqXexrXcIiOdRfJZbOJQ'}],
 'clusterDescription': 'Created with create_RDB notebook',
 'capacityConfiguration': {'nodeType': 'kx.s.xlarge', 'nodeCo

In [8]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME)
print("** DONE **")

** DONE **


In [9]:
cdf = get_clusters(client, ENV_ID, clusterType="RDB")

display(cdf)

clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
RDB_basictickdb,RUNNING,RDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'tphostfile', 'value': 'tickerplant'}]",Created with create_RDB notebook,2023-06-19 23:41:42.054000+00:00,2023-06-19 23:29:05.897000+00:00,basictickdb,[]


In [10]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-06-19 23:41:46.575495
